In [1]:
import os
import sys

# 1. Point to your environment's Library\bin (where FFmpeg DLLs live)
# Replace 'whisper4' with your actual new environment name if different
env_bin_path = r"C:\Users\pavlo\miniconda3\envs\whisper4\Library\bin"
if os.path.exists(env_bin_path):
    os.add_dll_directory(env_bin_path)

# 2. Point to the torchcodec folder itself
codec_path = r"C:\Users\pavlo\miniconda3\envs\whisper4\Lib\site-packages\torchcodec"
if os.path.exists(codec_path):
    os.add_dll_directory(codec_path)

In [ ]:
#%pip install --upgrade transformers datasets[audio] accelerate

  Using cached datasets-4.6.1-py3-none-any.whl.metadata (19 kB)
Using cached datasets-4.6.1-py3-none-any.whl (520 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 4.6.0
    Uninstalling datasets-4.6.0:
      Successfully uninstalled datasets-4.6.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Add this to downgrade torchcodec to 0.8.1
%pip install torchcodec==0.8.1

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install sounddevice
%pip install soundfile
#Install the silero VAD model for voice activity detection
%pip install silero-vad

Note: you may need to restart the kernel to use updated packages.
  Using cached soundfile-0.13.1-py2.py3-none-win_amd64.whl.metadata (16 kB)
Using cached soundfile-0.13.1-py2.py3-none-win_amd64.whl (1.0 MB)
Note: you may need to restart the kernel to use updated packages.
  Using cached silero_vad-6.2.1-py3-none-any.whl.metadata (9.5 kB)
Using cached silero_vad-6.2.1-py3-none-any.whl (9.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import sounddevice as sd
import queue
import threading
from scipy.signal import resample_poly
from math import gcd
import soundfile as so
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
import numpy as np

c:\Users\pavlo\miniconda3\envs\whisper5\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import soundfile as so
import os
from pathlib import Path

dll_path = Path(r"C:\Users\pavlo\miniconda3\Library\bin")

if dll_path.exists():
    os.add_dll_directory(str(dll_path))
else:
    print(f"Warning: {dll_path} not found. Check your Miniconda path.")

In [ ]:
# Trying out Whisper tiny from: https://huggingface.co/openai/whisper-large-v3
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-tiny"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)
device_for_pipeline = 0 if torch.cuda.is_available() else -1
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device_for_pipeline,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation",streaming=True)
sample = next(iter(dataset))
audio_sample=sample["audio"]
if isinstance(audio_sample,dict) and "array" in audio_sample:
    audio_input = audio_sample["array"]
elif isinstance(audio_sample,dict) and "path" in audio_sample:
    audio_input = so.read(audio_sample["path"])
else:
    audio_input = audio_sample
result = pipe(audio_input,return_timestamps=True)
print(result["text"])

c:\Users\pavlo\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pavlo\.cache\huggingface\hub\models--openai--whisper-tiny. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 167/167 [00:00<00:00, 578.70it/s, Materializing param=model.encoder.layers.3.self_attn_la

 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similarly drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Latins' work is really Greek after all, and can discover in it but little of Rocky Ithaca. Lennils, pictures, are a sort of upguards and atom paintings, and Mason's exquisite itals are as national as a jingo poem. Mr. Birkut Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampoo and a turkish bath. Next man.


In [4]:
import sys, torch, os
print("python:", sys.executable)
print("torch:", torch.__version__)
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

python: c:\Users\pavlo\miniconda3\envs\whisper5\python.exe
torch: 2.9.1+cu130
CUDA_VISIBLE_DEVICES: None


In [5]:
fc = torch.version.cuda
print(fc)

13.0


In [6]:
mf_s=44100 #Standard microphone sampling rate
tf_s=16000 #Silero required sampling rate; https://github.com/snakers4/silero-vad/wiki/Performance-Metrics#silero-vad-performance-metrics
chunk_size = 32.0 #ms (nominal chunk size for silero (512 chunks at 16kHz); https://github.com/snakers4/silero-vad/wiki/Performance-Metrics#silero-vad-performance-metrics)
nsm=int(mf_s*chunk_size/1000)#Number of samples in each chunk with the microphone sampling rate (1323 samples at 44.1kHz)
nst=int(tf_s*chunk_size/1000)#Number of samples in each chunk with the target sampling rate (480 samples at 16kHz)
min_speech_duration = 0.25 # 250ms minimum speech to be valid
VAD_threshold = 0.5 #Silero-VAD's output is a probability of voice presence in the current chunk. The probability value is thresholded to determine a speech classification (https://github.com/snakers4/silero-vad/wiki/Quality-Metrics#probability).
st=2.0#Silence time threshold in seconds; if the VAD detects silence for this amount of time, it will consider the speech segment to be complete
maxd=10.0#Maximum duration of a speech segment in seconds; if the VAD detects speech for this amount of time, it will consider the speech segment to be complete and buffer will be flushed

#Load models
model_id = "openai/whisper-tiny"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)    
processor = AutoProcessor.from_pretrained(model_id)
#device_for_pipeline = 0 if torch.cuda.is_available() else -1
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)
#Initialize VAD https://github.com/snakers4/silero-vad
torch.set_num_threads(1)
vad, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad')
vad.eval()#Set the VAD to evaluation mode
(get_speech_timestamps, save_audio, read_audio, VADIterator, collect_chunks) = utils#Extract the utility functions from the silero VAD utils https://medium.com/@aidenkoh/how-to-implement-high-speed-voice-recognition-in-chatbot-systems-with-whisperx-silero-vad-cdd45ea30904
# Initialize VADIterator
vad_iterator = VADIterator(vad, sampling_rate=tf_s, threshold=VAD_threshold)

aq=queue.Queue()#Audio queue for communication between the audio callback and the main thread
sbuf=[]#Speech buffer to store the current speech segment
scount=0#Silence counter
speaking=False#Flag to indicate if the user is currently speaking

sb=int(st*(1000/chunk_size))#Number of consecutive silent chunks required to consider the speech segment complete
spb=int(maxd*(1000/chunk_size))#Number of consecutive speech chunks required to consider the speech segment complete
min_spb = int(min_speech_duration * (1000 / chunk_size)) # Minimum chunks required for chunk to be considered valid speech segment

#Function to add audio samples to queue
def inq(insamples, frames, time_info, status):
    if status:
        print(status)
    aq.put(insamples)#Add the incoming audio samples to the queue (queue method: https://www.geeksforgeeks.org/python/queue-in-python/)
#Function to resample audio samples from the microphone sampling rate to the target sampling rate required by the VAD model
def resample(inaudio):
    seq=inaudio.flatten().astype(np.float32) #Get the audio samples from the queue
    #Resampling is input f*(tf_s/mf_s), i.e. mf_s*(tf_s/mf_s)=tf_s=16KHz. Better than using scipy.signal.resample, which adds samples and might introduce artifacts or aliasing effects. This allows for more control over frequencies. 
    resampled = resample_poly(seq, tf_s, mf_s)#Resample the audio samples to the target sampling rate using polyphase filtering (up then down conversion with appropriate anti-aliasing filtering after down conversion) (scipy method: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.resample_poly.html)
    return resampled.astype('float32')#Return the resampled audio samples as float32 (required by silero VAD; https://medium.com/@aidenkoh/how-to-implement-high-speed-voice-recognition-in-chatbot-systems-with-whisperx-silero-vad-cdd45ea30904)

#Silero-VAD inference function
def VAD_prob(sample_16k):
    with torch.no_grad():#Disable gradient calculation for inference
        audio_tensor = torch.from_numpy(sample_16k).float().unsqueeze(0)#Convert the resampled audio samples to a PyTorch tensor and add a batch dimension
        #prob = vad(audio_tensor, tf_s).item()#Get the VAD probability of voice presence in the current chunk (sampling rate not passed as keyword argument detected after error:Error in process_audio: RuntimeError: forward() is missing value for argument 'sr')
        vad_iterator(audio_tensor, return_seconds=True)
    return 1.0 if vad_iterator.triggered else 0.0#Detect if threshold is triggered and return 1.0 for speech and 0.0 for silence based on the VAD iterator's triggered state 

#Whisper inference function
def transcribe(speech_Segment):
    audio_a=np.concatenate(speech_Segment, axis=0)#Concatenate the audio chunks in the speech buffer to form a complete speech segment
    audio_a=audio_a/(np.max(np.abs(audio_a))+1e-6)#Normalize the audio samples to the range [-1, 1] to prevent clipping and ensure consistent volume levels 
    result = pipe(audio_a)#Transcribe the speech segment using the Whisper model and get the transcription result with timestamps
    #print(result["text"])#Print the transcribed text from the speech segment
    return result["text"]#Return the transcribed text from the speech segment
    
#Process audio chunks from the queue and perform VAD and transcription
def process_audio():
    global scount, speaking, sbuf
    while True:
        audio = aq.get()#Get the next audio chunk from the queue
        if audio is None:#If the audio chunk is None, it means the audio stream has ended, so the loop is exited
            break
        resampled_16=resample(audio)#Resample the audio chunk to the target sampling rate (16KHz) required by the VAD model
        if len(resampled_16) < nst:#If the resampled audio chunk is shorter than the expected number of samples for the VAD model, it is padded with zeros to ensure consistent input size
            resampled_16 = np.pad(resampled_16, (0, nst - len(resampled_16)))
        if len(resampled_16) > nst:#If the resampled audio chunk is longer than the expected number of samples for the VAD model, it is truncated to ensure the target number of samples for the VAD
            resampled_16 = resampled_16[:nst]
        prob = VAD_prob(resampled_16)#Get the VAD probability of voice presence in the current chunk
        if prob:#If the VAD probability exceeds the threshold, it is classified as speech
            sbuf.append(resampled_16)#The resampled audio chunk is added to the speech buffer
            scount=0#The silence counter is set to zero since speech is detected
            speaking=True#Set the speaking flag to True since speech is detected
        else:#If the VAD probability does not exceed the threshold, it is classified as silence
            if speaking: #If the speaker was previously speaking, the silence counter is incremented
                scount += 1
                if scount >= sb or len(sbuf) >= spb:#If the silence counter exceeds the silence threshold or the speech segments exceed the maximum number of speech segments
                    if len(sbuf) >= min_spb:#If the number of speech segments in the buffer exceeds the minimum required for a valid speech segment, it is transcribed
                        print(f"Speech segment lasting {len(sbuf)*(maxd/1000):.1f}s complete. Transcribing...")
                        res = transcribe(sbuf)#The current speech segment in the speech buffer is transcribed using the Whisper model
                        sbuf=[]#The speech buffer is cleared to prepare for the next speech segment
                        scount=0#The silence counter is reset to zero
                        speaking=False#Set the speaking flag to False since the speech segment is complete
                        print(f"[Transcript]: {res}")#Print the transcribed text from the speech segment
                    else:#If the number of speech segments in the buffer does not exceed the minimum required for a valid speech segment, it is discarded
                        scount=0#The silence counter is reset to zero
                        speaking=False#Set the speaking flag to False since the speech segment is discarded
                        print(f"Discarded speech segment lasting {len(sbuf)*(maxd/1000):.1f}s due to insufficient duration.")#Print a message indicating that the speech segment was discarded due to insufficient duration
                        sbuf=[]#The speech buffer is cleared to discard the current speech segment
#Real-time transcription in main
def main():
    print("Starting real-time transcription. Press Ctrl+C to stop.")
    #Threads: https://www.geeksforgeeks.org/python/multithreading-python-set-1/
    audio_thread = threading.Thread(target=process_audio)#Create a thread to process the audio chunks from the queue and perform VAD and transcription
    audio_thread.start()#Start the audio processing thread
    #Exception of keyboard interrupt to stop the audio-streaming and transcription 
    try:
        with sd.InputStream(samplerate=mf_s, channels=1, callback=inq, blocksize=nsm, dtype='float32'):#Start an audio input stream with the specified sampling rate, number of 
            #channels, callback function, block size, and data type
            #https://python-sounddevice.readthedocs.io/en/0.4.1/usage.html
            #Explanation of why the main thread needs to be kept alive: The audio stream runs in a separate thread and continuously captures audio data from the microphone.
            #If the main thread were to be terminated, the audio stream would also be stopped, and the real-time transcription would not work. By keeping the main thread alive, 
            # we ensure that the audio stream continues to run and 
            # captures audio data for processing by the VAD and Whisper models.
            while True:
                sd.sleep(1000)#Keep the main thread alive while the audio stream is active
    except KeyboardInterrupt: #https://www.geeksforgeeks.org/python/how-to-catch-a-keyboardinterrupt-in-python/
        print("Stopping real-time transcription...")
        aq.put(None)#Signal the audio processing thread to stop by putting None in the queue
        audio_thread.join()#Wait for the audio processing thread to finish https://www.geeksforgeeks.org/python/multithreading-python-set-1/
if __name__ == "__main__":
    main()

`torch_dtype` is deprecated! Use `dtype` instead!
Loading weights: 100%|██████████| 167/167 [00:00<00:00, 339.18it/s, Materializing param=model.encoder.layers.3.self_attn_layer_norm.weight]  
`torch_dtype` is deprecated! Use `dtype` instead!
Using cache found in C:\Users\pavlo/.cache\torch\hub\snakers4_silero-vad_master


Starting real-time transcription. Press Ctrl+C to stop.


Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


Speech segment lasting 1.4s complete. Transcribing...


A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> to see related `.generate()` flags.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> to see related `.generate()` flags.


[Transcript]:  with the staff in the student rest. I'm letting my answer switch on right now. I know.
Speech segment lasting 0.3s complete. Transcribing...
[Transcript]:  austerity.
Speech segment lasting 0.4s complete. Transcribing...
[Transcript]:  This is austerity.
Speech segment lasting 0.6s complete. Transcribing...
[Transcript]:  I know, I know, I'm here.
Speech segment lasting 0.7s complete. Transcribing...
[Transcript]:  This is so old. Where are we going now?
Speech segment lasting 0.5s complete. Transcribing...
[Transcript]:  want to define me this box.
Speech segment lasting 0.7s complete. Transcribing...
[Transcript]:  Please find this red pillow.
Speech segment lasting 0.6s complete. Transcribing...
[Transcript]:  where my shoes in the scene.
Speech segment lasting 0.1s complete. Transcribing...
[Transcript]:  Thank you.
Speech segment lasting 0.3s complete. Transcribing...
[Transcript]:  I don't need that.


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Speech segment lasting 0.5s complete. Transcribing...
[Transcript]:  I will need some point.
Speech segment lasting 0.3s complete. Transcribing...
[Transcript]:  He's bare with me.
Speech segment lasting 0.6s complete. Transcribing...
[Transcript]:  Please bear with me.
Speech segment lasting 0.3s complete. Transcribing...
[Transcript]:  I will help you.
Stopping real-time transcription...
